In [1]:
import pandas as pd
import numpy as np
import re
import textract

In [2]:
import tabula

In [3]:
#extraigo texto
archivo= 'tresalia\Bancos\Banamex_nacional\pdf\Accival PDF - Actual_TEXTO.pdf'

In [4]:
text = textract.process(archivo)

In [5]:
#cambio formato
encoding = 'utf-8'
text_str=text.decode(encoding)

In [6]:
#separo
pdf=re.split(r"[~\r\n]+", text_str)

In [7]:
information_pdf=pd.DataFrame(pdf).rename(columns={0:"informacion"})
information_pdf.head()

,informacion
0,7/10/2020
1,Posición Histórica
2,cffibanamex@ Casa de Bolsa
3,CONTRATO: RFC : CORTE AL: PERIODO:
4,9999999 5 KT


In [8]:
pdf

['7/10/2020',
 'Posición Histórica',
 'cffibanamex@ Casa de Bolsa',
 'CONTRATO: RFC : CORTE AL: PERIODO:',
 '9999999 5 KT',
 'XXXX999999XX9 30/SEP/2020',
 '01/SEP/2020 al 30/SEP/2020',
 'VALOR TOTAL DE LA CARTERA $4,525.55',
 'PERFIL: NO APLICA (P7)',
 'PORTAFOLIO DE INVERSIÓN',
 'Número de Titulas',
 'Emisora, Serie y Cuoón',
 'Monto de Inversión',
 'Fecha Pzo Tasa o',
 'Inicio',
 'Partic',
 'Fecha Vencto',
 'Costo Promedio',
 'Costo· Fiscal',
 'POSICION DEL MES ACTUAL (30/SEP/2020)',
 'Instrumentos de Deuda',
 'Depósito plazo fijo',
 '1 DEPPF-F 58331 090',
 '4,525.56 30/09/2020',
 'SUBTOTAL',
 '1 3.520 01/10/2020',
 'Efectivo Efectivo',
 'SUBTOTAL',
 'POSICION DEL MES ANTERIOR (31/AGO/2020)',
 'Instrumentos de Deuda',
 'Gubernam-reporto',
 '2,272 BONOS 210610',
 '4,043.99 31/08/2020',
 'SUBTOTAL',
 'Efectivo Efectivo',
 'SUBTOTAL',
 'TOTAL',
 '1 4.250 01/09/2020',
 '103.012320',
 'TOTAL',
 'NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM INT COLONIA DELEGACIÓN C.P. 00000 CIUDAD DE ME.

In [9]:
periodo_lista=[]
for i in range(len(pdf)):
    if re.search('PERIODO', pdf[i]):
        periodo_lista.append(pdf[i])
        periodo_lista.append(pdf[i+1])
        periodo_lista.append(pdf[i+2])
        periodo_lista.append(pdf[i+3])
periodo_lista
        

['CONTRATO: RFC : CORTE AL: PERIODO:',
 '9999999 5 KT',
 'XXXX999999XX9 30/SEP/2020',
 '01/SEP/2020 al 30/SEP/2020',
 'DETALLE DE MOVIMIENTOS DEL PERIODO',
 'Día Día Ope Liq',
 'Concepto',
 'MaOldVoIMAIEntNeTrOioSr',
 'Las comisiones que se muestran en el mismo renglón de las operaciones de compra o venta de productos financieros en la sección de DETALLE DE MOVIMIENTOS DEL PERIODO de este estado de cuenta, bajo el rubro de ?Comisión? hacen referencia a las comisiones cobradas por corretaje y están sumarizadas en el número 1 Total de Comisiones y Gastos de Intermediación.',
 'La forma para calcular el monto de las comisiones y gastos lo puede encontrar en la Gula de Servicios de Inversión en el sitio www.banamex.com/sitios/casa-de--bolsa/index.htrnl Todas las comisiones son sujetas de !VA.',
 'Para mayor información con respecto a las comisiones cobradas, favor de comunicarse con su promotor.',
 'DUPLICADO SIN VALIDEZCIFRAS EXPRESADAS EN PESOS']

In [10]:
contrato= periodo_lista[1]
rfc= periodo_lista[2].split(' ')[0]
periodo=periodo_lista[3]
datos=dict(zip(['contrato','rfc','periodo'],[contrato, rfc, periodo]))
datos

{'contrato': '9999999 5 KT',
 'rfc': 'XXXX999999XX9',
 'periodo': '01/SEP/2020 al 30/SEP/2020'}

In [11]:
#datos personales
datos_lista=[]
for i in pdf:
    if re.search('COLONIA',i):
        datos_lista.append(i)
        
datos_personales=datos_lista[0]
datos_personales
    

'NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM INT COLONIA DELEGACIÓN C.P. 00000 CIUDAD DE ME.XICO, MEXICO C.R. 11002'

In [12]:
tabla=tabula.read_pdf(archivo, pages='all')

In [13]:
#si la tabla tiene una columna llamada emisora
for i in tabla:
    for j in i:
        if re.search('emisora',j.lower()):
            pag3=i

In [14]:
pag3_1=(pag3.fillna(0)).drop(['Unnamed: 0'], axis=1)

In [15]:
pag3_1.rename(columns={'Día Día Concepto':'Día Concepto y Ope Liq', 'Emisora, Serie':'Emisora, Serie y Cupón',
                       'Abonos':'Abonos y Cargos'}, inplace=True)

In [16]:
pagina_3=(pag3_1.drop(pag3_1.index[[0,1]])).reset_index(drop=True)

In [17]:
#se puede exportar la pagina3
pagina_3

,Día Concepto y Ope Liq,"Emisora, Serie y Cupón",Títulos,Precio,Pzo,Tasa,Importe,Intereses,Comisión',Impuesto,Abonos y Cargos,Saldo
0,aldo Anterior a Favor. ..,0,0,0.000000,0.0,0.00,0,0.00,0.0,0,0,6.20
1,1 Vmto Reporto,BONOS 210610 000,"2,272",103.024481,0.0,4.25,"4,071.62",0.00,0.0,0,"4,071.62","4,077.82"
2,1 DiflSR.,BONOS 210610 000,0,0.000000,0.0,0.00,9.27-,0.00,0.0,R 9.27,9.27-,"4,068.55"
3,1 DiflSR.,BONOS 210610 000,0,0.000000,0.0,0.00,9.27-,0.00,0.0,R 9.27,9.27-,"4,059.28"
4,1 Inicio Reporto,BONOS 210610 000,"2,271",103.044238,1.0,4.23,"4,013.46-",0.00,0.0,0,"4,013.46-",45.82
...,...,...,...,...,...,...,...,...,...,...,...,...
57,25 Vmto Dep Plazo,DEPPF-F 54315 090,0,0.000000,0.0,3.54,"4,530.36",23.06,0.0,0,"4,530.36","4,530.36"
58,25 Inicio Dep Plazo,DEPPF-F 56358 090,0,0.000000,1.0,3.54,"4,530.36-",0.00,0.0,0,"4,530.36-",0.00
59,3( Vmto Dep Plazo,DEPPF-F 56358 090,0,0.000000,0.0,3.54,"4,553.42",23.06,0.0,0,"4,553.42","4,553.42"
60,3( Inicio Dep Plazo,DEPPF·F 58331 090,0,0.000000,1.0,3.52,"4,525.56-",0.00,0.0,0,"4,525.56-",27.86


In [18]:
dias=[]
for i in pagina_3['Día Concepto y Ope Liq']:
    if len(re.findall(r'\d+',i))>0:
        dias.append(re.findall(r'\d+',i)[0])
        
    else:
        dias.append('0')

In [19]:
dias_2=[dias[0]]
for i in range(1,len(dias)):
    if dias[i]=='0':
        dias_2.append(str(int(dias[i])+1))
    else:
        dias_2.append(dias[i])
                      

In [20]:
periodo=datos['periodo']

In [21]:
mes=re.findall(r'/\w{3}/\d+',periodo)[0]

In [22]:
fecha_lista=[]
for i in dias_2:
    if i=='0':
        fecha_lista.append('0')
    else:
        fecha_lista.append(i+mes)

In [23]:
pagina_3['fecha']=fecha_lista

In [24]:
pagina_3['titular']=datos_personales

In [25]:
pagina_3['banco']='banamex'

In [26]:
pagina_3['nacionalidad']='nacional'

In [27]:
if re.search('usd',text_str.lower()) or re.search('dolar', text_str.lower()) or re.search('dollar', text_str.lower()):
    moneda='usd'
else:
    moneda='mxn'

In [28]:
moneda

'mxn'

In [29]:
pagina_3['divisa']=moneda

In [30]:
pagina_3['fecha de corte']=periodo

In [31]:
pagina_3['cuenta']=datos['contrato']

In [32]:
pagina_3['saldo inicial']=pagina_3['Saldo'][0]

In [33]:
retiros=[]
for i in pagina_3['Abonos y Cargos']:
    if re.search(r'\d-$', str(i)):
        retiros.append(i)
    else:
        retiros.append('0')

In [34]:
pagina_3['retiro']=retiros

In [35]:
depositos=[]
for i in pagina_3['Abonos y Cargos']:
    if i not in retiros:
        depositos.append(i)
    else:
        depositos.append('0')

In [36]:
pagina_3['deposito']=depositos

In [37]:
pagina_3['saldo']=pagina_3['Saldo']

In [38]:
pagina_3['concepto']=pagina_3['Día Concepto y Ope Liq']

In [39]:
edo_cuenta= pagina_3[['banco','nacionalidad','fecha de corte', 'titular','cuenta','fecha','concepto','saldo inicial',
                    'retiro','deposito','saldo','divisa']]

In [40]:
edo_cuenta

,banco,nacionalidad,fecha de corte,titular,cuenta,fecha,concepto,saldo inicial,retiro,deposito,saldo,divisa
0,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,0,aldo Anterior a Favor. ..,6.20,0,0,6.20,mxn
1,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,1/SEP/2020,1 Vmto Reporto,6.20,0,"4,071.62","4,077.82",mxn
2,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,1/SEP/2020,1 DiflSR.,6.20,9.27-,0,"4,068.55",mxn
3,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,1/SEP/2020,1 DiflSR.,6.20,9.27-,0,"4,059.28",mxn
4,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,1/SEP/2020,1 Inicio Reporto,6.20,"4,013.46-",0,45.82,mxn
...,...,...,...,...,...,...,...,...,...,...,...,...
57,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,25/SEP/2020,25 Vmto Dep Plazo,6.20,0,"4,530.36","4,530.36",mxn
58,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,25/SEP/2020,25 Inicio Dep Plazo,6.20,"4,530.36-",0,0.00,mxn
59,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,3/SEP/2020,3( Vmto Dep Plazo,6.20,0,"4,553.42","4,553.42",mxn
60,banamex,nacional,01/SEP/2020 al 30/SEP/2020,NOMBRE APELLIDO P APELLIDO M CALLE NÚM EXT NÚM...,9999999 5 KT,3/SEP/2020,3( Inicio Dep Plazo,6.20,"4,525.56-",0,27.86,mxn


In [41]:
nombre_archivo=re.findall(r'\d+/\w+/\d+',periodo)[0]+'-banamex-casa-de-bolsa'
nombre_archivo=nombre_archivo.replace('/','-')
nombre_archivo

'01-SEP-2020-banamex-casa-de-bolsa'

In [42]:
#exporto
#edo_cuenta.to_csv('tresalia\Bancos\Banamex_nacional\salida\\'+nombre_archivo+'.txt' , sep='\t', index=False)